In [1]:
#!cp "/content/drive/MyDrive/Projects/dataset-scaled.zip" "./"
#!unzip -q -u "./dataset-scaled.zip" -d "./dataset/"
!ls -lah ./
#!nvidia-sli

total 40G
drwxrwxrwx 2 root root    0 May 27 19:37 .
drwxrwxrwx 2 root root    0 May 27 19:37 ..
-rwxrwxrwx 1 root root  316 May 27 19:38 .amlignore
-rwxrwxrwx 1 root root  316 May 27 19:38 .amlignore.amltmp
drwxrwxrwx 2 root root    0 May 27 19:38 .ipynb_aml_checkpoints
-rwxrwxrwx 1 root root  30K May 31 18:28 Pic2LUT.ipynb
-rwxrwxrwx 1 root root 3.3G May 27 19:39 dataset-7500--1.npz
-rwxrwxrwx 1 root root 3.3G May 31 17:59 dataset-7500--10.npz
-rwxrwxrwx 1 root root 3.3G May 31 18:04 dataset-7500--12.npz
-rwxrwxrwx 1 root root 3.3G May 31 18:10 dataset-7500--14.npz
-rwxrwxrwx 1 root root 3.3G May 31 18:16 dataset-7500--16.npz
-rwxrwxrwx 1 root root 286M May 31 18:21 dataset-7500--18.npz
-rwxrwxrwx 1 root root 3.3G May 27 19:40 dataset-7500--2.npz
-rwxrwxrwx 1 root root 3.3G May 27 19:49 dataset-7500--3.npz
-rwxrwxrwx 1 root root 3.3G May 27 20:00 dataset-7500--5.npz
-rwxrwxrwx 1 root root 3.3G May 27 20:02 dataset-7500--6.npz
-rwxrwxrwx 1 root root 3.3G May 27 20:13 dataset-7500--7.n

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)


import numpy as np
#import pandas as pd
import time
import sys
import os
import random
import gc

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

print(tf.__version__)
print(tf.config.list_physical_devices(), tf.test.is_gpu_available(),  tf.test.is_built_with_cuda())

2.3.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] True True


In [3]:
DATASET_BASEPATH = "./dataset-scaled"


#   Create train data
# get paths and zip
def load_files_from_folder(folderpath, filetype):
    all_files = []
    for root, dirs, files in os.walk(folderpath):
        for file in files:
            if file.endswith(filetype):
                file_path = os.path.join(root, file)
                all_files.append( (file_path, file) )
    return all_files

luts = load_files_from_folder(DATASET_BASEPATH, ".cube")
luts_db = {}
for l in luts:
    path, filename = l
    luts_db[filename] = path

images = load_files_from_folder(DATASET_BASEPATH, ".jpg")

train_paths = []
for img in images:
    path, filename = img
    try:
        lut_path = luts_db['lut-'+filename.split('.')[0]+'.cube']
        train_paths.append((path, lut_path))
    except Exception as e:
        print('didnt find lut', e)
print(len(train_paths))

0


In [4]:
# LUT TOOLS
def get_header(grid_size):
    return f"""
#LUT size
LUT_3D_SIZE {grid_size}

#data domain
DOMAIN_MIN 0.0 0.0 0.0
DOMAIN_MAX 1.0 1.0 1.0

#LUT data points
"""

def load_lut_text(path):
    with open(path) as fo:
        file = fo.readlines()
    
    file = [i.strip() for i in file]
    grid_size = -1
    firstlines = file[:64]
    for l in firstlines:
        if "LUT_3D_SIZE" in l:
            grid_size = int(l.strip().split(" ")[1])
            break
    assert grid_size > 0

    firstchars = []
    for c in file[:64]:
        if len(c) > 0:
            firstchars.append(c[0])
        else:
            firstchars.append(c)
    try:
        startpoint = min(firstchars.index('0'), firstchars.index('1'))
    except ValueError:
        startpoint = firstchars.index('0')

    return (file[startpoint:], grid_size)

def text_to_lut(txt):
    assert type(txt) != tuple, "You might be trying to put in a tuple"
    lut = []
    for l in txt:
        s = l.strip().split(" ")
        lut.append(tuple([float(i) for i in s]))
    return lut

def create_lut_file(lut, grid_size):
    txt = get_header(grid_size)
    for i in lut:
        s = f"{i[0]:.6f} {i[1]:.6f} {i[2]:.6f}\n"
        txt+=s
    return txt

def save_to_lut_file(lutstring, outputpath):
    with open(outputpath, "w") as fo:
        fo.write(lutstring)


In [6]:
del trainX
del trainY
del validX
del validY
gc.collect()

15405

In [5]:
# loading images and luts
try:
    del trainY
    del validX
    del validY
    gc.collect()
    time.sleep(0.5)
except:
    print("no trainX to delete")


trainX = []
trainY = []
from tqdm import tqdm
LIMIT_DATASET = 12_000 #
bad_shapes = 0
random.shuffle(train_paths) # Shuffle training set
for paths in tqdm(train_paths[-LIMIT_DATASET:]):
   img_path, lut_path = paths
   img = tf.keras.preprocessing.image.img_to_array( # not scaled to 0.0-1.0, do in model
       tf.keras.preprocessing.image.load_img(img_path)
   )

   lut = np.array(text_to_lut(
       load_lut_text(lut_path)[0]
   ))
   
   if lut.shape == (512,3):
       trainX.append(img)
       trainY.append(lut)
   else:
       bad_shapes += 1

print(bad_shapes, "bad shaped luts")

# Create validation data
VALIDATION_SPLIT_PERCENTAGE = 0.075
valid_split = int(len(trainX)*VALIDATION_SPLIT_PERCENTAGE)

# Create numpy arrays
validX = np.array(trainX[:valid_split])
validY = np.array(trainY[:valid_split])

trainX = np.array(trainX[valid_split:])
trainY = np.array(trainY[valid_split:])

trainX = trainX.reshape(trainX.shape[0], 240, 160, 3)
validX = validX.reshape(validX.shape[0], 240, 160, 3)
trainY = trainY.reshape(trainY.shape[0], trainY.shape[1] * trainY.shape[2])
validY = validY.reshape(validY.shape[0], validY.shape[1] * validY.shape[2])

gc.collect()

100%|██████████| 11000/11000 [09:25<00:00, 19.45it/s]
233 bad shaped luts


49

In [4]:
# Load from training data cache
BASE_PATH_CACHE = "./"

firsttime_bool = True
for files in load_files_from_folder(BASE_PATH_CACHE, ".npz"):
    path = files[0]
    print("Loading:", path)
    arrays = np.load(path)
    if firsttime_bool:
        trainX = arrays['arr_0'].astype('uint8')
        trainY = arrays['arr_1'].astype('float16')
        validX = arrays['arr_2'].astype('uint8')
        validY = arrays['arr_3'].astype('float16')
        firsttime_bool = False
    else:
        trainX = np.concatenate((arrays['arr_0'].astype('uint8'), trainX))
        trainY = np.concatenate((arrays['arr_1'].astype('float16'), trainY))
        validX = np.concatenate((arrays['arr_2'].astype('uint8'), validX))
        validY = np.concatenate((arrays['arr_3'].astype('float16'), validY))
        break
    del arrays
    gc.collect()
    print(trainX.shape, (sys.getsizeof(trainX)+sys.getsizeof(trainY))//1000000, "mb")

print(trainX.shape)

Loading: ./dataset-7500--1.npz
(6754, 240, 160, 3) 798 mb
Loading: ./dataset-7500--10.npz
(13521, 240, 160, 3)


In [ ]:
check = [(i.shape, i.dtype) for i in trainX]
for i in check:
    if i != check[0]:
        print("ERROR X")
check = [(i.shape, i.dtype) for i in trainY]
for i in check:
    if i != check[0]:
        print("ERROR Y")


NameError: name 'trainX' is not defined

In [28]:
import sys
gc.collect()
print("Size of training data:",(sys.getsizeof(trainX)+sys.getsizeof(trainY))//1000000,"mb")

Size of training data: 12859 mb


In [5]:
print(trainX.shape, trainY.shape)
print(validX.shape, validY.shape)

(13521, 240, 160, 3) (13521, 1536)
(1175, 240, 160, 3) (1175, 1536)


In [11]:
# Creating model
initializer = tf.random_normal_initializer(0., 0.02)
model = keras.Sequential()
model.add(layers.experimental.preprocessing.Rescaling(1.0 / 255, input_shape=(240,160,3)))
model.add(layers.Conv2D(filters=72, kernel_size=(3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_initializer='he_uniform'))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(units=1536,activation="relu"))
model.add(layers.Dropout(0.45))
model.add(layers.Dense(units=1536,activation="relu"))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(units=1536,activation="linear"))
'''
model.add(layers.experimental.preprocessing.Rescaling(1.0 / 255, input_shape=(240,160,3)))
#model.add(layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
#model.add(layers.BatchNormalization())
#model.add(layers.Conv2D(filters=24, kernel_size=(3,3), padding="same", activation="relu", kernel_initializer='he_uniform'))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
#model.add(layers.Conv2D(filters=4, kernel_size=(3,3), padding="same", activation="relu", kernel_initializer='he_uniform'))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))
#model.add(layers.BatchNormalization())
#model.add(layers.Conv2DTranspose(48, 4, strides=2,
#                                    padding='same',
#                                    kernel_initializer=initializer))
#model.add(layers.Conv2D(filters=48, kernel_size=(3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
#model.add(layers.BatchNormalization())
#model.add(layers.Conv2D(filters=3, kernel_size=(2,2), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.Flatten())
#model.add(layers.Dropout(0.25))
model.add(layers.Dense(units=1024, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(units=768, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(units=1536, activation="linear"))
'''
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_4 (Rescaling)      (None, 240, 160, 3)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 120, 80, 3)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 60, 40, 3)         0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 20, 3)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              1844224   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)             

In [12]:
NAME = f"simple-model-{int(time.time())}"
filename = NAME+"_model-{epoch:02d}-{val_accuracy:.3f}"
models_folder = "./trained/models/"
logs_folder = "./trained/logs"


# Load model
#model = None
#model = tf.keras.models.load_model('./trained/models/large2-model-1622493336_model-32-0.081.model')

opt = tf.keras.optimizers.Adam() # learning_rate=0.0005, decay=1e-6
model.compile(optimizer=opt, loss="mse", metrics="accuracy") 
checkpoint = ModelCheckpoint(models_folder+"{}.model".format(filename, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'), verbose=0,period=8) # saves only the best ones

tensorboard = TensorBoard(log_dir=logs_folder+"/{}".format(NAME), profile_batch=0)

print(trainX.shape, trainY.shape)

EPOCHS = 20
BATCH_SIZE = 100
print("Training: "+NAME) 
history = model.fit(x=trainX, y=trainY, batch_size=BATCH_SIZE, validation_data=(validX, validY), epochs=EPOCHS, callbacks=[tensorboard, checkpoint]) # Train

model.save(f"{models_folder}latest-{NAME}-e{EPOCHS}")


(13521, 240, 160, 3) (13521, 1536)
Training: simple-model-1622554860
Epoch 1/20
136/136 [==============================] - 2s 14ms/step - loss: 0.0464 - accuracy: 0.0090 - val_loss: 0.0261 - val_accuracy: 0.0136
Epoch 2/20
136/136 [==============================] - 1s 10ms/step - loss: 0.0209 - accuracy: 0.0190 - val_loss: 0.0184 - val_accuracy: 0.0706
Epoch 3/20
136/136 [==============================] - 1s 10ms/step - loss: 0.0199 - accuracy: 0.0207 - val_loss: 0.0182 - val_accuracy: 0.0672
Epoch 4/20
136/136 [==============================] - 1s 10ms/step - loss: 0.0197 - accuracy: 0.0209 - val_loss: 0.0183 - val_accuracy: 0.0213
Epoch 5/20
136/136 [==============================] - 1s 10ms/step - loss: 0.0197 - accuracy: 0.0202 - val_loss: 0.0184 - val_accuracy: 0.0681
Epoch 6/20
136/136 [===========

In [25]:
score = model.evaluate(validX, validY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.01597711257636547
Test accuracy: 0.08163265138864517
